<a href="https://colab.research.google.com/github/zeldaskywalker/more-than-surviving/blob/main/database_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PostgreSQL Result JSON Translation for TimelineJS

In [ ]:
import datetime

In [ ]:
mock_timeline_events_database_query_result = [
    (1, # ID from events table                                              0
     'title', # from events table                                           1
     datetime.datetime(2023, 3, 15), # start_date from events table         2
     'day', # start_date_accuracy from events table                         3
     datetime.datetime(2023, 3, 15), # from events table                    4
     'day', # end_date_accuracy from events table                           5
     'mock location name', # location_name from events table                6
     'mock short description', # shorts_description from events table       7
     ['mock', 'issue', 'types'], # issue_types from events table            8
     'mock_image_url.com', # url from images table                          9
     'mock image caption', # caption from images table                      10
     'mock image credit', # credit from images table                        11
     'mock image alternative text', # alt_text from images table            12
     'mock_thumbnail_link.com' # timeline_thumbnail_link from events table  13
     )]

In [ ]:
import json

def translate_sql_data_to_timeline_json(query_result_list):
  final_timeline_dict = {}
  final_events_list = []
  for row in query_result_list:
    event_dict = {}

    media_dict = {
        "url": row[9],
        "caption": row[10],
        "credit": row[11],
        "alt": row[12]
    }

    event_dict["media"] = media_dict

    start_date_accuracy = row[3]
    end_date_accuracy = row[5]

    start_date_year = row[2].strftime("%Y")
    end_date_year = row[4].strftime("%Y")

    # Determine correct start date format for JSON based on accuracy
    if start_date_accuracy == 'day':
      start_date_month = row[2].strftime("%m")
      start_date_day = row[2].strftime("%d")
      start_date_dict = {
          "year": start_date_year,
          "month": start_date_month,
          "day": start_date_day,
          }
    elif start_date_accuracy == 'month':
      start_date_month = row[2].strftime("%m")
      start_date_dict = {
          "year": start_date_year,
          "month": start_date_month,
          }
    else:
      start_date_dict = {
          "year": start_date_year,
          }

    event_dict["start_date"] = start_date_dict

    # Determine correct end date format for JSON based on accuracy
    if end_date_accuracy == 'day':
      end_date_month = row[4].strftime("%m")
      end_date_day = row[4].strftime("%d")
      end_date_dict = {
          "year": end_date_year,
          "month": end_date_month,
          "day": end_date_day,
          }
    elif end_date_accuracy == 'month':
      end_date_month = row[4].strftime("%m")
      end_date_dict = {
          "year": end_date_year,
          "month": end_date_month,
          }
    else:
      end_date_dict = {
          "year": end_date_year,
          }

    event_dict["end_date"] = end_date_dict

    text_dict = {
        "headline": row[1],
        "text": row[7],
    }

    event_dict["group"] = 1

    final_events_list.append(event_dict)

  final_timeline_dict["events"] = final_events_list
  final_timeline_json_string = json.dumps(final_timeline_dict)
  final_timeline_json = json.loads(final_timeline_json_string)
  return final_timeline_json

In [ ]:
translate_sql_data_to_timeline_json(mock_timeline_events_database_query_result)

{'events': [{'media': {'url': 'mock_image_url.com',
    'caption': 'mock image caption',
    'credit': 'mock image credit',
    'alt': 'mock image alternative text'},
   'start_date': {'year': '2023', 'month': '03', 'day': '15'},
   'end_date': {'year': '2023', 'month': '03', 'day': '15'},
   'group': 1}]}

# PostgreSQL Result JSON Translation for MapBox GL JS

In [ ]:
mock_map_events_database_query_result = [
    (1, # ID from events table                                              0
     'title', # from events table                                           1
     41.2774353842556, # longitude from events table                        2
     -70.098462995243, # latitude from events table                         3
     'mock location name', # location_name from events table                4
     'mock short description', # shorts_description from events table       5
     ['mock', 'issue', 'types'], # issue_types from events table            6
     'mock_thumbnail_link.com', # timeline_thumbnail_link from events table 7
     ['mock_activist_id'], # activists_ids from events table                8
     )]

In [ ]:
import json

def translate_sql_data_to_geo_json(query_result_list):
  final_mapbox_dict = {"type": "FeatureCollection"}
  final_features_list = []
  for row in query_result_list:
    feature_dict = {"type": "Feature"}

    properties_dict = {
        "id": row[0],
        "title": row[1],
        "description": row[5],
        "issue_types": row[6],
        "thumbnail": row[7],
        "activists": row[8],
        "location": row[4]
    }

    feature_dict["properties"] = properties_dict

    geometry_dict = {
        "type": "Point",
        "coordinates": [row[3], row[2]]
    }

    feature_dict["geometry"] = geometry_dict

    final_features_list.append(feature_dict)

  final_mapbox_dict["features"] = final_features_list
  final_mapbox_geojson_string = json.dumps(final_mapbox_dict)
  final_mapbox_geojson = json.loads(final_mapbox_geojson_string)
  return final_mapbox_geojson

In [ ]:
translate_sql_data_to_geo_json(mock_map_events_database_query_result)

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'id': 1,
    'title': 'title',
    'description': 'mock short description',
    'issue_types': ['mock', 'issue', 'types'],
    'thumbnail': 'mock_thumbnail_link.com',
    'activists': ['mock_activist_id'],
    'location': 'mock location name'},
   'geometry': {'type': 'Point',
    'coordinates': [-70.098462995243, 41.2774353842556]}}]}

# Replace Data from Database
These methods are untested currently. Once finalized, they will be integrated into JSON translation methods above.

As it exists, the data from the sheets will completely replace the database data, in order to add, update, or delete any data.

In [ ]:
import pandas
import psycopg2

In [ ]:
def database_table_to_dataframe(table, connection):
  database_df = pandas.read_sql_table(table, connection)
  return database_df

## Read Google Sheets Data

In [ ]:
from google.colab import auth
import gspread
from google.auth import default

In [ ]:
def google_sheets_table_to_dataframe(sheet_name):
  # authenticating to google
  auth.authenticate_user()
  creds, _ = default()
  gc = gspread.authorize(creds)

  # defining worksheet
  spreadsheet = gc.open('MTS DATA')
  if sheet_name == 'activists':
    worksheet = spreadsheet.worksheet("ACTIVISTS")
  elif sheet_name == 'events':
    worksheet = spreadsheet.worksheet("EVENTS")
  elif sheet_name == 'images':
    worksheet = spreadsheet.worksheet("IMAGES")
  else:
    raise ValueError

  # get_all_values gives a list of rows
  rows = worksheet.get_all_values()

  # convert to a DataFrame 
  sheet_df = pandas.DataFrame(rows)

  # creating columns name
  sheet_df.columns = sheet_df.iloc[0]
  sheet_df = sheet_df.iloc[1:]
  sheet_df = sheet_df.drop(index=[1, 2]).reset_index(drop=True)
  return sheet_df

In [ ]:
google_sheets_table_to_dataframe("activists")

,,ID,NAME,EVENT IDs,IMAGE LINK,TRIBAL AFFILIATION,DATE OF BIRTH,LONGITUDE OF BIRTH,LATITUDE OF BIRTH,DATE OF DEATH,SHORT BIO,LONG BIO,CITATION
0,,ACT1,Absalom Boston,EVT1,https://drive.google.com/file/d/1mVNyr11i-qXMl...,Nantucket,1785-00-00,41.27743538,-70.098463,1855-00-00,Prominent Nantucket whaling captain and busine...,"Absalom Boston, the son of Seneca Boston, a fr...","White, Barbara Ann. “Black Activism Before the..."
1,,ACT2,William Apes,EVT6,,,,,,,,,
2,,ACT3,Solomon Attaquin,EVT5,,,,,,,,,
3,,ACT4,"Mary J. ""Polly"" Johnson",,,,,,,,,,
4,,ACT5,Cynthia Conet Attaquin,EVT5,,,,,,,,,
5,,ACT6,Zelda,,,,,,,,,,


In [ ]:
def write_dataframe_to_databse(sheets_df, table, connection):
  sheets_df.to_sql(table, connection, if_exists='replace')

# Playground

# PostgreSQL Result JSON Translation for TimelineJS

In [ ]:
import datetime

In [ ]:
mock_timeline_events_database_query_result = [
    (1, # ID from events table                                              0
     'title', # from events table                                           1
     datetime.datetime(2023, 3, 15), # start_date from events table         2
     'day', # start_date_accuracy from events table                         3
     datetime.datetime(2023, 3, 15), # from events table                    4
     'day', # end_date_accuracy from events table                           5
     'mock location name', # location_name from events table                6
     'mock short description', # shorts_description from events table       7
     ['mock', 'issue', 'types'], # issue_types from events table            8
     'mock_image_url.com', # url from images table                          9
     'mock image caption', # caption from images table                      10
     'mock image credit', # credit from images table                        11
     'mock image alternative text', # alt_text from images table            12
     'mock_thumbnail_link.com' # timeline_thumbnail_link from events table  13
     )]

In [ ]:
import json

def translate_sql_data_to_timeline_json(query_result_list):
  final_timeline_dict = {}
  final_events_list = []
  for row in query_result_list:
    event_dict = {}

    media_dict = {
        "url": row[9],
        "caption": row[10],
        "credit": row[11],
        "alt": row[12]
    }

    event_dict["media"] = media_dict

    start_date_accuracy = row[3]
    end_date_accuracy = row[5]

    start_date_year = row[2].strftime("%Y")
    end_date_year = row[4].strftime("%Y")

    # Determine correct start date format for JSON based on accuracy
    if start_date_accuracy == 'day':
      start_date_month = row[2].strftime("%m")
      start_date_day = row[2].strftime("%d")
      start_date_dict = {
          "year": start_date_year,
          "month": start_date_month,
          "day": start_date_day,
          }
    elif start_date_accuracy == 'month':
      start_date_month = row[2].strftime("%m")
      start_date_dict = {
          "year": start_date_year,
          "month": start_date_month,
          }
    else:
      start_date_dict = {
          "year": start_date_year,
          }

    event_dict["start_date"] = start_date_dict

    # Determine correct end date format for JSON based on accuracy
    if end_date_accuracy == 'day':
      end_date_month = row[4].strftime("%m")
      end_date_day = row[4].strftime("%d")
      end_date_dict = {
          "year": end_date_year,
          "month": end_date_month,
          "day": end_date_day,
          }
    elif end_date_accuracy == 'month':
      end_date_month = row[4].strftime("%m")
      end_date_dict = {
          "year": end_date_year,
          "month": end_date_month,
          }
    else:
      end_date_dict = {
          "year": end_date_year,
          }

    event_dict["end_date"] = end_date_dict

    text_dict = {
        "headline": row[1],
        "text": row[7],
    }

    event_dict["group"] = 1

    final_events_list.append(event_dict)

  final_timeline_dict["events"] = final_events_list
  final_timeline_json_string = json.dumps(final_timeline_dict)
  final_timeline_json = json.loads(final_timeline_json_string)
  return final_timeline_json

In [ ]:
translate_sql_data_to_timeline_json(mock_timeline_events_database_query_result)

{'events': [{'media': {'url': 'mock_image_url.com',
    'caption': 'mock image caption',
    'credit': 'mock image credit',
    'alt': 'mock image alternative text'},
   'start_date': {'year': '2023', 'month': '03', 'day': '15'},
   'end_date': {'year': '2023', 'month': '03', 'day': '15'},
   'group': 1}]}